<a href="https://colab.research.google.com/github/lachyjung/DACON_-/blob/master/prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1.batch size와 다른 예측 input
2.inverse minmax scaler
csv 파일로 저장하기


In [ ]:
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from torch import nn
from torch.nn import Module
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt

In [ ]:
train_features = pd.read_csv('/content/drive/My Drive/충돌체/KAERI_dataset/train_features.csv')
train_features = train_features.iloc[:,2:]

    
max_feature = train_features.max().max()
min_feature = train_features.min().min()
div_feature = max_feature - min_feature
# self.train_features = np.divide((self.train_features-self.min_feature),self.div_feature)

In [ ]:
class myModel(Module):
  def __init__(self, input_dim, hidden_dim, output_dim):
    super(myModel,self).__init__()
    self.lstm = nn.LSTM(input_dim, hidden_dim, 1,batch_first=True)
    self.fc_1 = nn.Linear(hidden_dim, 50, bias=True)
    self.fc_2= nn.Linear(50,20, bias=True)
    self.output = nn.Linear(20, output_dim, bias=True)
    
    self.relu = nn.ReLU()
    self.dropout = nn.Dropout(0.5)
    self.hidden_dim = hidden_dim
  

  def forward(self, inputs):
    lstm_out, hidden_cell = self.lstm(inputs, (torch.zeros(1,inputs.shape[0] ,self.hidden_dim), torch.zeros(1,inputs.shape[0] ,self.hidden_dim)))
    layer_1 = self.dropout(self.relu(self.fc_1(hidden_cell[0])))
    layer_2 = self.dropout(self.relu(self.fc_2(layer_1)))
    prediction = self.output(layer_2)
    return prediction


In [ ]:
class dataset(Dataset):
  def __init__(self,min_feature,max_feature):
    submission = pd.read_csv('/content/drive/My Drive/충돌체/KAERI_dataset/train_features.csv')
    submission = submission.iloc[-300:,2:]
    self.min_ = min_feature
    self.max_ = max_feature
    self.div_ = self.max_ - self.min_

    submission = np.divide((submission - self.min_), self.div_)
    
    self.feed_data = torch.from_numpy(np.array(submission)).float()
    
    
    submission_ = pd.read_csv('/content/drive/My Drive/충돌체/KAERI_dataset/train_target.csv')
    
    submission_ = np.divide((submission - self.min_), self.div_)
    submission = submission.iloc[-300:,1:]
    self.feed_data_ = torch.from_numpy(np.array(submission_)).float()
    
    self.window = 375

  def __getitem__(self, index):
    return self.feed_data[(self.window * index): (self.window * index)+ self.window], self.feed_data_[index]

  def __len__(self):
    return len(self.feed_data_)


In [ ]:
dataloader = DataLoader(dataset=dataset(min_feature, max_feature), batch_size=1, shuffle=False)

In [ ]:
model = myModel(4,20,4)

In [ ]:
state_dict = torch.load('/content/drive/My Drive/충돌체/model_history/best_model.pth')
print(state_dict.keys())

odict_keys(['lstm.weight_ih_l0', 'lstm.weight_hh_l0', 'lstm.bias_ih_l0', 'lstm.bias_hh_l0', 'fc_1.weight', 'fc_1.bias', 'fc_2.weight', 'fc_2.bias', 'output.weight', 'output.bias'])


In [ ]:
model.load_state_dict(state_dict)
loss = nn.MSELoss()

In [ ]:
loss_sum = []

for feature, target in dataloader:
  pred = model(feature)
  '''pred = torch.flatten(pred)
  pred = pred.detach().numpy()
  pred = (pred*div_feature) + min_feature


  target = torch.flatten(target)
  target = target.detach().numpy()
  target = (target*div_feature) + min_feature'''
  
  print(pred, target)


In [ ]:
print(np.array(loss_sum).mean())

nan


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: RuntimeWarning: Mean of empty slice.
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [ ]:
submit = pd.DataFrame(columns=['X','Y','M','V'])

for i,data in enumerate(dataloader):
  pred = model(data)
  pred = torch.flatten(pred)
  pred = pred.detach().numpy()
  pred = (pred*div_feature) + min_feature
  pred = list(pred)
  submit.loc[f'{i+2800}'] = pred
  print(i)



In [ ]:
print(submit.info())
print(submit.head(20))

<class 'pandas.core.frame.DataFrame'>
Index: 700 entries, 2800 to 3499
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   X       700 non-null    float64
 1   Y       700 non-null    float64
 2   M       700 non-null    float64
 3   V       700 non-null    float64
dtypes: float64(4)
memory usage: 27.3+ KB
None
             X         Y         M         V
2800 -176768.5 -182309.5 -110073.5  -52373.5
2801 -213585.0 -187108.0 -190038.0 -103805.0
2802 -424422.0    8936.0 -586959.0 -626141.0
2803 -238092.0 -194135.5 -455008.5 -324708.5
2804 -345148.0   54710.0 -560413.0 -369791.5
2805  134175.5  -45911.0 -200806.5  -48570.5
2806 -109267.5  194698.0   82105.5  -23802.5
2807  159446.0  -24005.5  -84863.0  275196.0
2808  593032.5  -52582.5  468021.0  113614.0
2809  -97371.5 -100532.0   55420.5 -213132.0
2810  327574.0 -236893.0  224262.5  -92152.0
2811    6638.0 -203553.5 -404571.0  -90886.5
2812  126569.0  -19232.0  286801.5  465333.

In [ ]:
submit.index.names = ['id']

In [ ]:
print(submit.head(20))

             X         Y         M         V
id                                          
2800 -176768.5 -182309.5 -110073.5  -52373.5
2801 -213585.0 -187108.0 -190038.0 -103805.0
2802 -424422.0    8936.0 -586959.0 -626141.0
2803 -238092.0 -194135.5 -455008.5 -324708.5
2804 -345148.0   54710.0 -560413.0 -369791.5
2805  134175.5  -45911.0 -200806.5  -48570.5
2806 -109267.5  194698.0   82105.5  -23802.5
2807  159446.0  -24005.5  -84863.0  275196.0
2808  593032.5  -52582.5  468021.0  113614.0
2809  -97371.5 -100532.0   55420.5 -213132.0
2810  327574.0 -236893.0  224262.5  -92152.0
2811    6638.0 -203553.5 -404571.0  -90886.5
2812  126569.0  -19232.0  286801.5  465333.0
2813 -218476.0  -51324.0  -37393.5 -416309.5
2814   70333.0  166581.0  323711.0   50945.0
2815  -83705.5   67756.5 -230136.0   85360.5
2816 -257032.5 -233866.5 -509329.5 -141600.0
2817 -354750.5 -205509.0 -496644.0 -301006.5
2818   69049.0   43998.5  -79756.0   90810.5
2819 -395090.0 -123320.5 -636286.0 -410324.0


In [ ]:
submit.to_csv('/content/drive/My Drive/충돌체/콕찔러_01.csv', sep=',', na_rep='NaN')